![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Clinical Relation Extraction Models

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp_jsl.version : 3.1.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

## 1. Posology Releation Extraction

This is a demonstration of using SparkNLP for extracting posology relations. The following relatios are supported:

DRUG-DOSAGE
DRUG-FREQUENCY
DRUG-ADE (Adversed Drug Events)
DRUG-FORM
DRUG-ROUTE
DRUG-DURATION
DRUG-REASON
DRUG=STRENGTH

The model has been validated agains the posology dataset described in (Magge, Scotch, & Gonzalez-Hernandez, 2018).

| Relation | Recall | Precision | F1 | F1 (Magge, Scotch, & Gonzalez-Hernandez, 2018) |
| --- | --- | --- | --- | --- |
| DRUG-ADE | 0.66 | 1.00 | **0.80** | 0.76 |
| DRUG-DOSAGE | 0.89 | 1.00 | **0.94** | 0.91 |
| DRUG-DURATION | 0.75 | 1.00 | **0.85** | 0.92 |
| DRUG-FORM | 0.88 | 1.00 | **0.94** | 0.95* |
| DRUG-FREQUENCY | 0.79 | 1.00 | **0.88** | 0.90 |
| DRUG-REASON | 0.60 | 1.00 | **0.75** | 0.70 |
| DRUG-ROUTE | 0.79 | 1.00 | **0.88** | 0.95* |
| DRUG-STRENGTH | 0.95 | 1.00 | **0.98** | 0.97 |


*Magge, Scotch, Gonzalez-Hernandez (2018) collapsed DRUG-FORM and DRUG-ROUTE into a single relation.

In [0]:
import re
import functools 
from scipy import spatial
import pyspark.sql.types as T

**Build pipeline using SparNLP pretrained models and the relation extration model optimized for posology**.
 
 The precision of the RE model is controlled by "setMaxSyntacticDistance(4)", which sets the maximum syntactic distance between named entities to 4. A larger value will improve recall at the expense at lower precision. A value of 4 leads to literally perfect precision (i.e. the model doesn't produce any false positives) and reasonably good recall.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = sparknlp.annotators.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

ner_tagger = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

ner_event = MedicalNerModel.pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags") 

ner_chunker = NerConverter()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel.pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
ner_events_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

**Create a light pipeline for annotating free text**

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

lmodel = LightPipeline(model)
results = lmodel.fullAnnotate(text)

In [0]:
results[0]['ner_chunks']

Out[14]: [Annotation(chunk, 28, 33, 1 unit, {'entity': 'DOSAGE', 'sentence': '0', 'chunk': '0'}),
 Annotation(chunk, 38, 42, Advil, {'entity': 'DRUG', 'sentence': '0', 'chunk': '1'}),
 Annotation(chunk, 44, 53, for 5 days, {'entity': 'DURATION', 'sentence': '0', 'chunk': '2'}),
 Annotation(chunk, 96, 101, 1 unit, {'entity': 'DOSAGE', 'sentence': '1', 'chunk': '3'}),
 Annotation(chunk, 106, 114, Metformin, {'entity': 'DRUG', 'sentence': '1', 'chunk': '4'}),
 Annotation(chunk, 116, 120, daily, {'entity': 'FREQUENCY', 'sentence': '1', 'chunk': '5'}),
 Annotation(chunk, 190, 197, 40 units, {'entity': 'DOSAGE', 'sentence': '2', 'chunk': '6'}),
 Annotation(chunk, 202, 217, insulin glargine, {'entity': 'DRUG', 'sentence': '2', 'chunk': '7'}),
 Annotation(chunk, 219, 226, at night, {'entity': 'FREQUENCY', 'sentence': '2', 'chunk': '8'}),
 Annotation(chunk, 231, 238, 12 units, {'entity': 'DOSAGE', 'sentence': '2', 'chunk': '9'}),
 Annotation(chunk, 243, 256, insulin lispro, {'entity': 'DRUG', 'sentence': '2', 'chunk': '10'}),
 Annotation(chunk, 258, 267, with meals, {'entity': 'FREQUENCY', 'sentence': '2', 'chunk': '11'}),
 Annotation(chunk, 275, 283, metformin, {'entity': 'DRUG', 'sentence': '2', 'chunk': '12'}),
 Annotation(chunk, 285, 291, 1000 mg, {'entity': 'STRENGTH', 'sentence': '2', 'chunk': '13'}),
 Annotation(chunk, 293, 307, two times a day, {'entity': 'FREQUENCY', 'sentence': '2', 'chunk': '14'})]

In [0]:
results[0]['relations']

Out[15]: [Annotation(category, 28, 42, DOSAGE-DRUG, {'chunk2': 'Advil', 'confidence': '1.0', 'entity2_end': '42', 'chunk1': '1 unit', 'entity2_begin': '38', 'entity1': 'DOSAGE', 'entity1_begin': '28', 'entity1_end': '33', 'entity2': 'DRUG'}),
 Annotation(category, 38, 53, DRUG-DURATION, {'chunk2': 'for 5 days', 'confidence': '1.0', 'entity2_end': '53', 'chunk1': 'Advil', 'entity2_begin': '44', 'entity1': 'DRUG', 'entity1_begin': '38', 'entity1_end': '42', 'entity2': 'DURATION'}),
 Annotation(category, 96, 114, DOSAGE-DRUG, {'chunk2': 'Metformin', 'confidence': '1.0', 'entity2_end': '114', 'chunk1': '1 unit', 'entity2_begin': '106', 'entity1': 'DOSAGE', 'entity1_begin': '96', 'entity1_end': '101', 'entity2': 'DRUG'}),
 Annotation(category, 106, 120, DRUG-FREQUENCY, {'chunk2': 'daily', 'confidence': '1.0', 'entity2_end': '120', 'chunk1': 'Metformin', 'entity2_begin': '116', 'entity1': 'DRUG', 'entity1_begin': '106', 'entity1_end': '114', 'entity2': 'FREQUENCY'}),
 Annotation(category, 190, 217, DOSAGE-DRUG, {'chunk2': 'insulin glargine', 'confidence': '1.0', 'entity2_end': '217', 'chunk1': '40 units', 'entity2_begin': '202', 'entity1': 'DOSAGE', 'entity1_begin': '190', 'entity1_end': '197', 'entity2': 'DRUG'}),
 Annotation(category, 202, 226, DRUG-FREQUENCY, {'chunk2': 'at night', 'confidence': '1.0', 'entity2_end': '226', 'chunk1': 'insulin glargine', 'entity2_begin': '219', 'entity1': 'DRUG', 'entity1_begin': '202', 'entity1_end': '217', 'entity2': 'FREQUENCY'}),
 Annotation(category, 231, 256, DOSAGE-DRUG, {'chunk2': 'insulin lispro', 'confidence': '1.0', 'entity2_end': '256', 'chunk1': '12 units', 'entity2_begin': '243', 'entity1': 'DOSAGE', 'entity1_begin': '231', 'entity1_end': '238', 'entity2': 'DRUG'}),
 Annotation(category, 243, 267, DRUG-FREQUENCY, {'chunk2': 'with meals', 'confidence': '1.0', 'entity2_end': '267', 'chunk1': 'insulin lispro', 'entity2_begin': '258', 'entity1': 'DRUG', 'entity1_begin': '243', 'entity1_end': '256', 'entity2': 'FREQUENCY'}),
 Annotation(category, 275, 291, DRUG-STRENGTH, {'chunk2': '1000 mg', 'confidence': '1.0', 'entity2_end': '291', 'chunk1': 'metformin', 'entity2_begin': '285', 'entity1': 'DRUG', 'entity1_begin': '275', 'entity1_end': '283', 'entity2': 'STRENGTH'}),
 Annotation(category, 275, 307, DRUG-FREQUENCY, {'chunk2': 'two times a day', 'confidence': '1.0', 'entity2_end': '307', 'chunk1': 'metformin', 'entity2_begin': '293', 'entity1': 'DRUG', 'entity1_begin': '275', 'entity1_end': '283', 'entity2': 'FREQUENCY'})]

**Show extracted relations**

In [0]:
for rel in results[0]["relations"]:
    print("{}({}={} - {}={})".format(
        rel.result, 
        rel.metadata['entity1'], 
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['chunk2']
    ))

DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Advil)
DRUG-DURATION(DRUG=Advil - DURATION=for 5 days)
DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Metformin)
DRUG-FREQUENCY(DRUG=Metformin - FREQUENCY=daily)
DOSAGE-DRUG(DOSAGE=40 units - DRUG=insulin glargine)
DRUG-FREQUENCY(DRUG=insulin glargine - FREQUENCY=at night)
DOSAGE-DRUG(DOSAGE=12 units - DRUG=insulin lispro)
DRUG-FREQUENCY(DRUG=insulin lispro - FREQUENCY=with meals)
DRUG-STRENGTH(DRUG=metformin - STRENGTH=1000 mg)
DRUG-FREQUENCY(DRUG=metformin - FREQUENCY=two times a day)

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df


rel_df = get_relations_df (results)

rel_df

Out[17]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DOSAGE-DRUG,DOSAGE,28,33,1 unit,DRUG,38,42,Advil,1.0
1,DRUG-DURATION,DRUG,38,42,Advil,DURATION,44,53,for 5 days,1.0
2,DOSAGE-DRUG,DOSAGE,96,101,1 unit,DRUG,106,114,Metformin,1.0
3,DRUG-FREQUENCY,DRUG,106,114,Metformin,FREQUENCY,116,120,daily,1.0
4,DOSAGE-DRUG,DOSAGE,190,197,40 units,DRUG,202,217,insulin glargine,1.0
5,DRUG-FREQUENCY,DRUG,202,217,insulin glargine,FREQUENCY,219,226,at night,1.0
6,DOSAGE-DRUG,DOSAGE,231,238,12 units,DRUG,243,256,insulin lispro,1.0
7,DRUG-FREQUENCY,DRUG,243,256,insulin lispro,FREQUENCY,258,267,with meals,1.0
8,DRUG-STRENGTH,DRUG,275,283,metformin,STRENGTH,285,291,1000 mg,1.0
9,DRUG-FREQUENCY,DRUG,275,283,metformin,FREQUENCY,293,307,two times a day,1.0


In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df



Out[18]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DURATION-DRUG,DURATION,493,500,five-day,DRUG,512,522,amoxicillin,1.0
1,DRUG-DURATION,DRUG,681,693,dapagliflozin,DURATION,695,708,for six months,1.0
2,DRUG-ROUTE,DRUG,1940,1946,insulin,ROUTE,1948,1951,drip,1.0
3,DOSAGE-DRUG,DOSAGE,2255,2262,40 units,DRUG,2267,2282,insulin glargine,1.0
4,DRUG-FREQUENCY,DRUG,2267,2282,insulin glargine,FREQUENCY,2284,2291,at night,1.0
5,DOSAGE-DRUG,DOSAGE,2295,2302,12 units,DRUG,2307,2320,insulin lispro,1.0
6,DRUG-FREQUENCY,DRUG,2307,2320,insulin lispro,FREQUENCY,2322,2331,with meals,1.0
7,DRUG-STRENGTH,DRUG,2339,2347,metformin,STRENGTH,2349,2355,1000 mg,1.0
8,DRUG-FREQUENCY,DRUG,2339,2347,metformin,FREQUENCY,2357,2371,two times a day,1.0


### Visualization of Extracted Relations

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and was discharged on 40 units of insulin glargine , 
12 units of insulin lispro with meals , and metformin two times a day.
"""

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
from sparknlp_display import RelationExtractionVisualizer


visualizer = RelationExtractionVisualizer()
vis = visualizer.display(results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

/databricks/python/lib/python3.8/site-packages/sparknlp_display/relation_extraction.py:366: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
 relation_coordinates = np.array(relation_coordinates)

The patient was prescribed 1 unit DOSAGE of Advil DRUG for 5 days DURATION after meals. The patient was also given 1 unit DOSAGE of Metformin DRUG daily FREQUENCY . He was seen by the endocrinology service and was discharged on 40 units DOSAGE of insulin glargine DRUG , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG two times a day FREQUENCY . <polyline fill="none" points="333.0,155.0 332.11224489795916,153.7960798646822 331.2244897959183,152.59312871337622 330.3367346938776,151.39211553009378 329.44897959183675,150.19400929884657 328.5612244897959,148.99977900364644 327.67346938775506,147.81039362850512 326.78571428571433,146.62682215743445 325.8979591836735,145.45003357444605 325.01020408163265,144.28099686355176 324.1224489795918,143.12068100876337 323.234693877551,141.97005499409258 322.34693877551024,140.83008780355124 321.4591836734694,139.70174842115105 320.57142857142856,138.5860058309038 319.6836734693878,137.48382901682123 318.7959183673469,136.39618696291512 317.90816326530614,135.32404865319725 317.0204081632653,134.2683830716793 316.1326530612245,133.23015920237316 315.2448979591836,132.2103460292905 314.3571428571428,131.20991253644314 313.46938775510205,130.22982770784282 312.5816326530612,129.2710605275013 311.6938775510204,128.33457997943034 310.80612244897964,127.42135504764173 309.9183673469388,126.53235471614718 309.03061224489795,125.6685479689585 308.14285714285717,124.83090379008749 307.2551020408163,124.02039116354581 306.36734693877554,123.2379790733453 305.4795918367347,122.48463650349771 304.59183673469386,121.76133243801479 303.7040816326531,121.06903586090831 302.81632653061223,120.40871575619002 301.92857142857144,119.78134110787173 301.0408163265306,119.18788089996517 300.1530612244898,118.62930411648207 299.265306122449,118.10657974143427 298.3775510204082,117.62067675883348 297.48979591836735,117.17256415269148 296.60204081632656,116.76321090702004 295.7142857142857,116.39358600583091 294.8265306122449,116.06465843313586 293.9387755102041,115.77739717294665 293.05102040816325,115.53277120927505 292.16326530612247,115.33174952613282 291.2755102040816,115.17530110753174 290.38775510204084,115.06439493748353 289.5,115.0 289.5,115.0 288.61224489795916,114.9827622844223 287.7244897959184,115.01203580140927 286.83673469387753,115.08685156694916 285.94897959183675,115.20624059703015 285.0612244897959,115.36923390764053 284.17346938775506,115.57486251476851 283.28571428571433,115.82215743440234 282.39795918367355,116.11014968253025 281.51020408163276,116.43787027514047 280.6224489795918,116.80435022822121 279.73469387755097,117.20862055776078 278.8469387755101,117.64971227974738 277.95918367346934,118.12665641016923 277.07142857142856,118.63848396501459 276.1836734693878,119.18422596027166 275.2959183673469,119.7629134119287 274.40816326530614,120.37357733597398 273.5204081632653,121.01524874839565 272.63265306122446,121.68695866518203 271.7448979591836,122.3877381023213 270.85714285714283,123.11661807580174 269.96938775510205,123.87262960161156 269.0816326530612,124.654803695739 268.1938775510204,125.46217137417233 267.3061224489796,126.29376365289971 266.41836734693874,127.14861154790944 265.53061224489795,128.02574607518974 264.64285714285717,128.92419825072886 263.7551020408163,129.842999090515 262.86734693877554,130.7811796105364 261.97959183673476,131.73777082678137 261.09183673469386,132.71180375523804 260.2040816326531,133.7023094118947 259.31632653061223,134.7083188127396 258.42857142857144,135.72886297376095 257.54081632653066,136.76297291094698 256.6530612244898,137.80967964028594 255.76530612244898,138.86801417776604 254.87755102040816,139.9370075393756 253.98979591836735,141.01569074110276 253.10204081632654,142.1030947989358 252.21428571428572,143.19825072886297 251.32653061224488,144.30018954687245 250.4387755102041,145.40794226895255 249.55102040816325,146.52053991109145 248.66326530612247,147.63701348927742 247.77551020408166,148.75639401949866 246.887

## 2. Clinical RE

**The set of relations defined in the 2010 i2b2 relation challenge**

TrIP: A certain treatment has improved or cured a medical problem (eg, ‘infection resolved with antibiotic course’)

TrWP: A patient's medical problem has deteriorated or worsened because of or in spite of a treatment being administered (eg, ‘the tumor was growing despite the drain’)

TrCP: A treatment caused a medical problem (eg, ‘penicillin causes a rash’)

TrAP: A treatment administered for a medical problem (eg, ‘Dexamphetamine for narcolepsy’)

TrNAP: The administration of a treatment was avoided because of a medical problem (eg, ‘Ralafen which is contra-indicated because of ulcers’)

TeRP: A test has revealed some medical problem (eg, ‘an echocardiogram revealed a pericardial effusion’)

TeCP: A test was performed to investigate a medical problem (eg, ‘chest x-ray done to rule out pneumonia’)

PIP: Two problems are related to each other (eg, ‘Azotemia presumed secondary to sepsis’)

In [0]:
clinical_ner_tagger = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel.pretrained("re_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setRelationPairs(["problem-test", "problem-treatment"]) # we can set the possible relation pairs (if not set, all the relations will be calculated)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
re_clinical download started this may take some time.
Approximate size to download 6 MB
[ | ][ / ][OK!]

In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.relation!="O"]


Out[22]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
11,TrAP,PROBLEM,617,620,T2DM,TREATMENT,626,637,atorvastatin,0.99955326
15,TeRP,TEST,739,758,Physical examination,PROBLEM,796,810,dry oral mucosa,0.9994142
19,TrWP,TEST,1246,1258,blood samples,PROBLEM,1265,1274,hemolyzing,0.9854173
21,TeRP,TEST,1535,1547,the anion gap,PROBLEM,1553,1566,still elevated,0.9965193
26,TrAP,TEST,1838,1845,analysis,PROBLEM,1854,1880,interference from turbidity,0.9676019
33,TrAP,PROBLEM,1967,1969,HTG,TREATMENT,1976,1986,a reduction,0.9875973
34,TrAP,PROBLEM,1967,1969,HTG,TEST,1991,2003,the anion gap,0.9993911


## 3. Clinical Temporal Events RE

Temporal relations, or temporal links (denoted by the TLINK tag), indicate whether and how two EVENTs, two TIME, or an EVENT and a TIME related to each other in the clinical timeline. There are 3 type of relations here and below are some examples of Relations, with square brackets indicating EVENT and TIME connected by a temporal link:

**`BEFORE`**

The patient was given stress dose steroids prior to his surgery. ([stress dose steroids] `BEFORE` [his surgery])

The patient had an undocumented history of possible atrial fibrillation prior to admission. ([possible atrial fibrillation] `BEFORE` [admission])

His nasogastric tube was discontinued on 05-26-98. ([His nasogastric] `BEFORE` [05-26-98])

**`AFTER`**

Before admission, he had another serious concussion. ([admission] `AFTER` [another serious concussion])

On postoperative day No 1, he was started on Percocet. ([Percocet] `AFTER` [postoperative day No 1])


**`OVERLAP`**

She denies any fevers or chills. ([fevers] `OVERLAP` [chills])

The patient's serum creatinine on discharge date, 2012-05-06, was 1.9. ([discharge date] `OVERLAP` [2012-05-06])

His preoperative workup was completed and included a normal white count ([a normal white count] `OVERLAP` [His preoperative workup])

The patient had an undocumented history of possible atrial fibrillation prior to admission. ([possible atrial fibrillation] `OVERLAP` [admission])


| Relation | Recall | Precision | F1 |
| --- | --- | --- | --- |
| OVERLAP | 0.81 | 0.73 | **0.77** |
| BEFORE | 0.85 | 0.88 | **0.86** |
| AFTER | 0.38 | 0.46 | **0.43** |

This RE model works with `ner_events_clinical` NER model and expect the following entities as inputs:

[`OCCURRENCE`,
 `DATE`,
 `DURATION`,
 `EVIDENTIAL`,
 `TEST`,
 `PROBLEM`,
 `TREATMENT`,
 `CLINICAL_DEPT`,
 `FREQUENCY`,
 `TIME`]

In [0]:
events_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setPredictionThreshold(0.9)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
re_temporal_events_clinical download started this may take some time.
Approximate size to download 5.9 MB
[ | ][ / ][OK!]

In [0]:
events_ner_tagger.getClasses()

Out[24]: ['O',
 'B-OCCURRENCE',
 'B-DATE',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-DURATION',
 'I-DURATION',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'B-TREATMENT',
 'I-TREATMENT',
 'I-DATE',
 'B-TEST',
 'I-TEST',
 'B-CLINICAL_DEPT',
 'I-CLINICAL_DEPT',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'I-EVIDENTIAL',
 'B-TIME',
 'I-TIME']

In [0]:
text ="She is diagnosed as cancer in 1991. Then she was admitted to Mayo Clinic in May 2000 and discharged in October 2001"

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df

Out[25]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,BEFORE,PROBLEM,20,25,cancer,DATE,30,33,1991,0.9989215
1,AFTER,OCCURRENCE,49,56,admitted,CLINICAL_DEPT,61,71,Mayo Clinic,1.0
2,OVERLAP,OCCURRENCE,49,56,admitted,DATE,76,83,May 2000,0.99152386
3,BEFORE,CLINICAL_DEPT,61,71,Mayo Clinic,OCCURRENCE,89,98,discharged,1.0
4,BEFORE,OCCURRENCE,89,98,discharged,DATE,103,114,October 2001,0.95415086


In [0]:
text ="On 9–28-92, the patient will return for chemotherapy and she will follow up with her primary doctor, for PT and Coumadin dosing on Monday."

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df[(rel_df.relation!="O")&(rel_df.entity1!=rel_df.entity2)]


Out[26]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,OVERLAP,DATE,3,9,9–28-92,TREATMENT,40,51,chemotherapy,1.000000
1,OVERLAP,DATE,3,9,9–28-92,TREATMENT,105,106,PT,0.989851
3,OVERLAP,OCCURRENCE,66,74,follow up,CLINICAL_DEPT,81,98,her primary doctor,0.998086
4,OVERLAP,OCCURRENCE,66,74,follow up,TREATMENT,105,106,PT,0.999761
5,OVERLAP,OCCURRENCE,66,74,follow up,TREATMENT,112,126,Coumadin dosing,0.958726
6,OVERLAP,CLINICAL_DEPT,81,98,her primary doctor,TREATMENT,105,106,PT,1.000000
7,OVERLAP,CLINICAL_DEPT,81,98,her primary doctor,DATE,131,136,Monday,0.984193
9,AFTER,TREATMENT,105,106,PT,DATE,131,136,Monday,0.926052
10,OVERLAP,TREATMENT,112,126,Coumadin dosing,DATE,131,136,Monday,0.999999


**Admission-Discharge Events Exraction**

We can also extract `ADMISSION` and `DISCHARGE` entities in addition to `ner_events_clinical` model entities by using `ner_events_admission_clinical` NER model and expect the following entities as inputs:
   
[`EVIDENTIAL`,
 `OCCURRENCE`,
 `TREATMENT`,
 `DATE`,
 `ADMISSION`,
 `TIME`,
 `FREQUENCY`,
 `CLINICAL_DEPT`,
 `DURATION`,
 `PROBLEM`,
 `TEST`,
 `DISCHARGE`]

In [0]:
events_admission_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_admission_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_admission_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_admission_clinical download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][OK!]

In [0]:
events_admission_ner_tagger.getClasses()

Out[28]: ['O',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'I-TREATMENT',
 'I-DATE',
 'B-ADMISSION',
 'B-TREATMENT',
 'I-TIME',
 'I-FREQUENCY',
 'B-FREQUENCY',
 'B-OCCURRENCE',
 'I-CLINICAL_DEPT',
 'B-DURATION',
 'I-PROBLEM',
 'I-TEST',
 'B-DATE',
 'B-TIME',
 'I-EVIDENTIAL',
 'I-DURATION',
 'B-DISCHARGE',
 'B-TEST',
 'B-PROBLEM',
 'B-CLINICAL_DEPT']

In [0]:
text ="""She is admitted to The John Hopkins Hospital on Monday with a history of gestational diabetes mellitus diagnosed.  
She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 
12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge. 
"""

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]


Out[29]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,AFTER,ADMISSION,7,14,admitted,CLINICAL_DEPT,19,43,The John Hopkins Hospital,0.993847
1,OVERLAP,ADMISSION,7,14,admitted,DATE,48,53,Monday,0.991630
2,OVERLAP,CLINICAL_DEPT,19,43,The John Hopkins Hospital,DATE,48,53,Monday,1.000000
3,OVERLAP,DATE,48,53,Monday,PROBLEM,73,101,gestational diabetes mellitus,0.929829
4,OVERLAP,CLINICAL_DEPT,132,156,the endocrinology service,DATE,184,193,03/02/2018,0.999932
5,OVERLAP,DISCHARGE,170,179,discharged,DATE,184,193,03/02/2018,1.000000
6,AFTER,DISCHARGE,170,179,discharged,TREATMENT,210,225,insulin glargine,0.999233
7,AFTER,DISCHARGE,170,179,discharged,TREATMENT,241,254,insulin lispro,0.997469
8,OVERLAP,DATE,184,193,03/02/2018,TREATMENT,210,225,insulin glargine,0.999975
9,OVERLAP,DATE,184,193,03/02/2018,TREATMENT,241,254,insulin lispro,0.999682


**Test-Result-Date Extraction**

There is another Relation Extraction model to link clinical tests to test results and dates to clinical entities: `re_test_result_date`

In [0]:
trd_ner_tagger = MedicalNerModel()\
    .pretrained("jsl_ner_wip_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")       

trd_re_Model = RelationExtractionModel()\
    .pretrained("re_test_result_date", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    trd_ner_tagger,
    ner_chunker,
    dependency_parser,
    trd_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

jsl_ner_wip_clinical download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][OK!]
re_test_result_date download started this may take some time.
Approximate size to download 9.3 MB
[ | ][ / ][OK!]

In [0]:
text = 'Hospitalized with pneumonia on 11 June 2019, confirmed by a positive PCR of any specimen, evidenced by SPO2 </= 93%.'

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]

Out[31]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,is_date_of,Disease_Syndrome_Disorder,18,26,pneumonia,Date,31,42,11 June 2019,0.998061
3,is_finding_of,Disease_Syndrome_Disorder,18,26,pneumonia,O2_Saturation,103,114,SPO2 </= 93%,0.984967
6,is_date_of,Date,31,42,11 June 2019,O2_Saturation,103,114,SPO2 </= 93%,0.993791
7,is_result_of,Test_Result,60,67,positive,Test,69,71,PCR,1.000000
8,is_result_of,Test_Result,60,67,positive,O2_Saturation,103,114,SPO2 </= 93%,1.000000


## 4. Human Phenotype - Gene RE

https://github.com/lasigeBioTM/PGR

Human phenotype-gene relations are fundamental to fully understand the origin of some phenotypic abnormalities and their associated diseases. Biomedical literature is the most comprehensive source of these relations, however, we need Relation Extraction tools to automatically recognize them. We present the Phenotype-Gene Relations (PGR) model, trained on a silver standard corpus of human phenotype and gene annotations and their relations. 

It extracts 2 label: `True` or `False`

In [0]:
pgr_ner_tagger = MedicalNerModel()\
    .pretrained("ner_human_phenotype_gene_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pgr_re_Model = RelationExtractionModel()\
    .pretrained("re_human_phenotype_gene_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["hp-gene",'gene-hp'])\
    .setMaxSyntacticDistance(4)\

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    pgr_ner_tagger,
    ner_chunker,
    dependency_parser,
    pgr_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_human_phenotype_gene_clinical download started this may take some time.
Approximate size to download 14 MB
[ | ][ / ][OK!]
re_human_phenotype_gene_clinical download started this may take some time.
Approximate size to download 5.9 MB
[ | ][ / ][OK!]

In [0]:
text = "She has a retinal degeneration, hearing loss and renal failure, short stature, \
Mutations in the SH3PXD2B gene coding for the Tks4 protein are responsible for the autosomal recessive."

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!=0)]

rel_df

Out[33]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,HP,32,43,hearing loss,GENE,96,103,SH3PXD2B,0.99999976
1,1,HP,49,61,renal failure,GENE,96,103,SH3PXD2B,0.9998969
2,1,HP,64,76,short stature,GENE,96,103,SH3PXD2B,0.99994695


## 5. Drug-Drug Interaction RE

In clinical application, two or more drugs are often used in combination to achieve conducive results, such as synergistic effect, increasing therapeutic effect and reducing or delaying the occurrence of
drug resistance. However, there is a potential for harmful drug-drug
interactions (DDIs) to occur when two or more drugs are taken at the
same time or at certain interval, which can reduce or invalidate the
efficacy of drugs, and increase toxicity or even cause death. Therefore,
in order to prevent harmful drug-drug interaction (DDI), medical staff
often spend much time in reviewing the relevant drug alert literature
and drug knowledge bases. 

**ref**: *Drug-drug interaction extraction via hybrid neural networks on biomedical literature
https://www-sciencedirect-com.ezproxy.leidenuniv.nl:2443/science/article/pii/S1532046420300605*

In [0]:
ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_tags"]) \
    .setOutputCol("ner_chunk")

ddi_re_model = RelationExtractionModel()\
    .pretrained("re_drug_drug_interaction_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["drug-drug"])\
    .setMaxSyntacticDistance(4)\

ddi_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_converter,
    dependency_parser,
    ddi_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
ddi_model = ddi_pipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
re_drug_drug_interaction_clinical download started this may take some time.
Approximate size to download 4.6 MB
[ | ][ / ][OK!]

In [0]:
text='When carbamazepine is withdrawn from the combination therapy, aripiprazole dose should then be reduced. \
If additional adrenergic drugs are to be administered by any route, \
they should be used with caution because the pharmacologically predictable sympathetic effects of Metformin may be potentiated'

lmodel = LightPipeline(ddi_model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)


rel_df

Out[35]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DDI-advise,DRUG,5,17,carbamazepine,DRUG,62,73,aripiprazole,0.99238014


In [0]:
annotations[0]['ner_chunk']

Out[36]: [Annotation(chunk, 5, 17, carbamazepine, {'entity': 'DRUG', 'sentence': '0', 'chunk': '0', 'confidence': '0.9968'}),
 Annotation(chunk, 62, 73, aripiprazole, {'entity': 'DRUG', 'sentence': '0', 'chunk': '1', 'confidence': '0.9998'}),
 Annotation(chunk, 118, 133, adrenergic drugs, {'entity': 'DRUG', 'sentence': '1', 'chunk': '2', 'confidence': '0.7984'}),
 Annotation(chunk, 270, 278, Metformin, {'entity': 'DRUG', 'sentence': '1', 'chunk': '3', 'confidence': '0.9998'})]

## 6. Chemical–Protein Interactions (ChemProt RE)

Accurately detecting the interactions between chemicals and proteins is a crucial task that plays a key role in precision medicine, drug discovery and basic clinical research. Currently, PubMed contains >28 million articles, and its annual growth rate is more than a million articles each year. A large amount of valuable chemical–protein interactions (CPIs) are hidden in the biomedical literature. There is an increasing interest in CPI extraction from the biomedical literature.

Since manually extracting biomedical relations such as protein–protein interactions (PPI) and drug–drug interactions (DDI) is costly and time-consuming, some computational methods have been successfully proposed for automatic biomedical relation extraction.

To date, most studies on the biomedical relation extraction have focused on the PPIs and DDIs, but a few attempts have been made to extract CPIs. The BioCreative VI ChemProt shared task released the ChemProt dataset for CPI extraction, which is the first challenge for extracting CPIs.

Computational CPI extraction is generally approached as a task of classifying whether a specified semantic relation holds between the chemical and protein entities within a sentence or document. The ChemProt corpus is a manually annotated CPI dataset, which greatly promotes the development of CPI extraction approaches. 

ref: https://academic.oup.com/database/article/doi/10.1093/database/baz054/5498050

| Relation | Recall | Precision | F1 | F1 (Zhang, Yijia, et al., 2019) |
| --- | --- | --- | --- | --- |
| CPR:3 | 0.47 | 0.59 | **0.52** | 0.594 |
| CPR:4 | 0.72 | 0.81 | **0.77** | 0.718 |
| CPR:5 | 0.43 | 0.88 | **0.58** | 0.657 |
| CPR:6 | 0.59 | 0.89 | **0.71** | 0.725 |
| CPR:9 | 0.62 | 0.84 | **0.71** | 0.501 |
|avg. |  |  | **0.66** | 0.64 |

ChemProt RE works well with `ner_chemprot_clinical` find relationships between the following entities

`CHEMICAL`: Chemical entity mention type; 

`GENE-Y`: gene/protein mention type that can be normalized or associated to a biological database identifier; 

`GENE-N`: gene/protein mention type that cannot be normalized to a database identifier.

In [0]:
ner_tagger = MedicalNerModel()\
    .pretrained("ner_chemprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_tags"]) \
    .setOutputCol("ner_chunk")

chemprot_re_model = RelationExtractionModel()\
    .pretrained("re_chemprot_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\

chemprot_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_converter,
    dependency_parser,
    chemprot_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
chemprot_model = chemprot_pipeline.fit(empty_data)


ner_chemprot_clinical download started this may take some time.
Approximate size to download 15.3 MB
[ | ][ / ][OK!]
re_chemprot_clinical download started this may take some time.
Approximate size to download 5.9 MB
[ | ][ / ][OK!]

In [0]:
text='''
In this study, we examined the effects of mitiglinide on various cloned K(ATP) channels (Kir6.2/SUR1, Kir6.2/SUR2A, and Kir6.2/SUR2B) reconstituted in COS-1 cells, and compared them to another meglitinide-related compound, nateglinide. Patch-clamp analysis using inside-out recording configuration showed that mitiglinide inhibits the Kir6.2/SUR1 channel currents in a dose-dependent manner (IC50 value, 100 nM) but does not significantly inhibit either Kir6.2/SUR2A or Kir6.2/SUR2B channel currents even at high doses (more than 10 microM). Nateglinide inhibits Kir6.2/SUR1 and Kir6.2/SUR2B channels at 100 nM, and inhibits Kir6.2/SUR2A channels at high concentrations (1 microM). Binding experiments on mitiglinide, nateglinide, and repaglinide to SUR1 expressed in COS-1 cells revealed that they inhibit the binding of [3H]glibenclamide to SUR1 (IC50 values: mitiglinide, 280 nM; nateglinide, 8 microM; repaglinide, 1.6 microM), suggesting that they all share a glibenclamide binding site. The insulin responses to glucose, mitiglinide, tolbutamide, and glibenclamide in MIN6 cells after chronic mitiglinide, nateglinide, or repaglinide treatment were comparable to those after chronic tolbutamide and glibenclamide treatment. These results indicate that, similar to the sulfonylureas, mitiglinide is highly specific to the Kir6.2/SUR1 complex, i.e., the pancreatic beta-cell K(ATP) channel, and suggest that mitiglinide may be a clinically useful anti-diabetic drug.
'''


lmodel = LightPipeline(chemprot_model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.entity1!=rel_df.entity2]

Out[38]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,CPR:2,CHEMICAL,43,53,mitiglinide,GENE-N,80,87,channels,0.6070414
1,CPR:2,GENE-N,80,87,channels,CHEMICAL,224,234,nateglinide,0.98933965
4,CPR:4,CHEMICAL,706,716,mitiglinide,GENE-Y,751,754,SUR1,0.7594982
6,CPR:2,CHEMICAL,719,729,nateglinide,GENE-Y,751,754,SUR1,0.9999553
7,CPR:2,CHEMICAL,736,746,repaglinide,GENE-Y,751,754,SUR1,0.9919803
8,CPR:2,CHEMICAL,823,839,[3H]glibenclamide,GENE-Y,844,847,SUR1,1.0
11,CPR:2,GENE-N,998,1004,insulin,CHEMICAL,1019,1025,glucose,0.59488183
12,CPR:2,GENE-N,998,1004,insulin,CHEMICAL,1028,1038,mitiglinide,0.5599123
13,CPR:3,GENE-N,998,1004,insulin,CHEMICAL,1041,1051,tolbutamide,0.98802537
14,CPR:3,GENE-N,998,1004,insulin,CHEMICAL,1058,1070,glibenclamide,0.98900056


# Train a Relation Extraction Model

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_clinical_rel_dataset.csv

dbutils.fs.cp("file:/databricks/driver/i2b2_clinical_rel_dataset.csv", "dbfs:/")


Out[39]: True

In [0]:
# if you need to customize the DL arcitecture (more layers, more features etc.)

'''
import tensorflow as tf

from sparknlp_jsl.training import tf_graph

#%tensorflow_version 1.x

tf_graph.build("relation_extraction", build_params={"input_dim": 6000, "output_dim": 3, 'batch_norm':1, "hidden_layers": [300, 200], "hidden_act": "relu", 'hidden_act_l2':1}, model_location=".", model_filename="re_with_BN")
'''

Out[40]: '\nimport tensorflow as tf\n\nfrom sparknlp_jsl.training import tf_graph\n\n#%tensorflow_version 1.x\n\ntf_graph.build("relation_extraction", build_params={"input_dim": 6000, "output_dim": 3, \'batch_norm\':1, "hidden_layers": [300, 200], "hidden_act": "relu", \'hidden_act_l2\':1}, model_location=".", model_filename="re_with_BN")\n'

In [0]:
# tf_graph.print_model_params("relation_extraction")

In [0]:
data = spark.read.option("header","true").format("csv").load("/i2b2_clinical_rel_dataset.csv")

data = data.select( 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel','dataset')

data.show(10)

# you only need these columns>> 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel'
# ('dataset' column is optional)

+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
 sentence|firstCharEnt1|firstCharEnt2|lastCharEnt1|lastCharEnt2| chunk1| chunk2| label1| label2| rel|dataset|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
VITAL SIGNS - Tem...| 49| 75| 64| 84| respiratory rate| saturation| test| test| O| test|
No lotions , crea...| 3| 34| 9| 42| lotions| incisions|treatment| problem|TrNAP| test|
Because of expect...| 11| 58| 54| 68|expected long ter...| a picc line|treatment|treatment| O| train|
She states this l...| 16| 82| 31| 92| light-headedness| diaphoresis| problem| problem| PIP| train|
Initial electroca...| 196| 145| 239| 176|an inferior and r...|1-mm st depressio...| problem| problem| PIP| test|
Abdominal ultraso...| 1| 54| 20| 71|abdominal ultrasound| gallbladder sludge| test| problem| TeRP| test|
At that time he w...| 99| 139| 133| 173|ir placement of a...|his abdominopelvi...|treatment| problem| TrAP| test|
He was transferre...| 143| 195| 154| 213| reintubation| worsening pneumonia|treatment| problem| TrAP| test|
5) Anemia - previ...| 12| 35| 23| 47| previous w/u| elev ferritin| test| problem| TeRP| test|
The patient &apos...| 161| 149| 179| 156| po pain medications| his pain|treatment| problem| TrAP| train|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
only showing top 10 rows

In [0]:
data.groupby('dataset').count().show()

+-------+-----+
dataset|count|
+-------+-----+
 train| 350|
 test| 650|
+-------+-----+

In [0]:
#Annotation structure
annotationType = T.StructType([
            T.StructField('annotatorType', T.StringType(), False),
            T.StructField('begin', T.IntegerType(), False),
            T.StructField('end', T.IntegerType(), False),
            T.StructField('result', T.StringType(), False),
            T.StructField('metadata', T.MapType(T.StringType(), T.StringType()), False),
            T.StructField('embeddings', T.ArrayType(T.FloatType()), False)
        ])

#UDF function to convert train data to names entitities

@F.udf(T.ArrayType(annotationType))
def createTrainAnnotations(begin1, end1, begin2, end2, chunk1, chunk2, label1, label2):
    
    entity1 = sparknlp.annotation.Annotation("chunk", begin1, end1, chunk1, {'entity': label1.upper(), 'sentence': '0'}, [])
    entity2 = sparknlp.annotation.Annotation("chunk", begin2, end2, chunk2, {'entity': label2.upper(), 'sentence': '0'}, [])    
        
    entity1.annotatorType = "chunk"
    entity2.annotatorType = "chunk"

    return [entity1, entity2]    

#list of valid relations
rels = ["TrIP", "TrAP", "TeCP", "TrNAP", "TrCP", "PIP", "TrWP", "TeRP"]

#a query to select list of valid relations
valid_rel_query = "(" + " OR ".join(["rel = '{}'".format(rel) for rel in rels]) + ")"

data = data\
  .withColumn("begin1i", F.expr("cast(firstCharEnt1 AS Int)"))\
  .withColumn("end1i", F.expr("cast(lastCharEnt1 AS Int)"))\
  .withColumn("begin2i", F.expr("cast(firstCharEnt2 AS Int)"))\
  .withColumn("end2i", F.expr("cast(lastCharEnt2 AS Int)"))\
  .where("begin1i IS NOT NULL")\
  .where("end1i IS NOT NULL")\
  .where("begin2i IS NOT NULL")\
  .where("end2i IS NOT NULL")\
  .where(valid_rel_query)\
  .withColumn(
      "train_ner_chunks", 
      createTrainAnnotations(
          "begin1i", "end1i", "begin2i", "end2i", "chunk1", "chunk2", "label1", "label2"
      ).alias("train_ner_chunks", metadata={'annotatorType': "chunk"}))
    

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

In [0]:
!mkdir re_graph
!mkdir re_logs

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb -P /databricks/driver/re_graph/

In [0]:
%fs ls file:/databricks/driver/re_graph/

path,name,size
file:/databricks/driver/re_graph/RE_in1200D_out20.pb,RE_in1200D_out20.pb,135162


In [0]:
documenter = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")
    
dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

# set training params and upload model graph (see ../Healthcare/8.Generic_Classifier.ipynb)
reApproach = RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(50)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile("/databricks/driver/re_graph/RE_in1200D_out20.pb")\
    .setFixImbalance(True)\
    .setFromEntity("begin1i", "end1i", "label1")\
    .setToEntity("begin2i", "end2i", "label2")\
    .setOutputLogsPath('file:/databricks/driver/re_logs')

finisher = Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

train_pipeline = Pipeline(stages=[
    documenter, tokenizer, words_embedder, pos_tagger, 
    dependency_parser, reApproach, finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]

In [0]:
rel_model = train_pipeline.fit(train_data)

In [0]:
rel_model.stages[-2]

Out[54]: RelationExtractionModel_e85623a08f2a

In [0]:
rel_model.stages[-2].write().overwrite().save("dbfs:/databricks/driver/models/custom_RE_model")

In [0]:
result = rel_model.transform(test_data)

In [0]:
recall = result\
  .groupBy("rel")\
  .agg(F.avg(F.expr("IF(rel = relations_out, 1, 0)")).alias("recall"))\
  .select(
      F.col("rel").alias("relation"), 
      F.format_number("recall", 2).alias("recall"))\
  .show()

performance  = result\
  .where("relations_out <> ''")\
  .groupBy("relations_out")\
  .agg(F.avg(F.expr("IF(rel = relations_out, 1, 0)")).alias("precision"))\
  .select(
      F.col("relations_out").alias("relation"), 
      F.format_number("precision", 2).alias("precision"))\
  .show()

+--------+------+
relation|recall|
+--------+------+
 TrIP| 0.18|
 TrAP| 0.36|
 TeCP| 0.07|
 TrNAP| 0.11|
 TrCP| 0.00|
 PIP| 0.45|
 TrWP| 0.00|
 TeRP| 0.26|
+--------+------+

+--------+---------+
relation|precision|
+--------+---------+
 TrIP| 0.67|
 TrAP| 0.78|
 TeCP| 0.25|
 TrNAP| 0.05|
 TrCP| 0.00|
 PIP| 0.87|
 TeRP| 0.97|
+--------+---------+

In [0]:
result_df = result.select(F.explode(F.arrays_zip('relations.result', 'relations.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("relation"),
        F.expr("cols['1']['entity1']").alias("entity1"),
        F.expr("cols['1']['entity1_begin']").alias("entity1_begin"),
        F.expr("cols['1']['entity1_end']").alias("entity1_end"),
        F.expr("cols['1']['chunk1']").alias("chunk1"),
        F.expr("cols['1']['entity2']").alias("entity2"),
        F.expr("cols['1']['entity2_begin']").alias("entity2_begin"),
        F.expr("cols['1']['entity2_end']").alias("entity2_end"),
        F.expr("cols['1']['chunk2']").alias("chunk2"),
        F.expr("cols['1']['confidence']").alias("confidence")
        )

result_df.show(50, truncate=100)

+--------+---------+-------------+-----------+----------------------------------------------------------+-------+-------------+-----------+-----------------------------------------------------------+----------+
relation| entity1|entity1_begin|entity1_end| chunk1|entity2|entity2_begin|entity2_end| chunk2|confidence|
+--------+---------+-------------+-----------+----------------------------------------------------------+-------+-------------+-----------+-----------------------------------------------------------+----------+
 PIP| PROBLEM| 196| 239| an inferior and right ventricular infarction|PROBLEM| 145| 176| 1-mm st depressions in i and avl|0.85219336|
 TeRP| TEST| 1| 20| abdominal ultrasound|PROBLEM| 54| 71| gallbladder sludge|0.58730906|
 TrAP|TREATMENT| 99| 133| ir placement of a drainage catheter|PROBLEM| 139| 173| his abdominopelvic fluid collection| 0.8456566|
 TeRP| TEST| 1| 13| a colonoscopy|PROBLEM| 44| 51| bleeding|0.51441264|
 TeRP| TEST| 1| 8| a ct abd|PROBLEM| 16| 36| 8mm obstructing stone| 0.7644676|
 TeRP| TEST| 1| 9| pathology|PROBLEM| 19| 77|poorly differentiated squamous cell carcinoma of the cervix|0.63362217|
 TrAP|TREATMENT| 96| 99| cpap|PROBLEM| 57| 76| abdominal distention|0.59488845|
 TrNAP|TREATMENT| 119| 126| diuresis|PROBLEM| 1| 20| creatinine elevation|0.59645915|
 PIP| PROBLEM| 70| 89| right hydronephrosis|PROBLEM| 3| 52| an 8 x 7 x 7 mm stone in the proximal right ureter| 0.5685163|
 TrAP|TREATMENT| 10| 15| repair|PROBLEM| 20| 32| neck fracture|0.58386457|
 TrCP|TREATMENT| 36| 85| percutaneous endoscopic gastrostomy tube placement|PROBLEM| 3| 22| failed swallow study| 0.782331|
 TrAP|TREATMENT| 90| 99| management|PROBLEM| 104| 129| her pulmonary hypertension|0.53591317|
 PIP| PROBLEM| 54| 81| paroxysmal nocturnal dyspnea|PROBLEM| 14| 48| a new two to three-pillow orthopnea|0.81033355|
 TeRP| TEST| 1| 14| the ultrasound|PROBLEM| 22| 35| biliary sludge| 0.8062734|
 TrAP|TREATMENT| 98| 100| ddi|PROBLEM| 69| 80| pancytopenia|0.53061485|
 TrAP|TREATMENT| 14| 20| removal|PROBLEM| 25| 33| cbd stone| 0.5988157|
 PIP| PROBLEM| 166| 223|posterior cerebral artery distribution cerebral infarction|PROBLEM| 230| 240| mass effect|0.82680815|
 TrNAP| TEST| 10| 18| the study|PROBLEM| 112| 132| a cecal cystic lesion|0.55497926|
 PIP| PROBLEM| 35| 40| a boil|PROBLEM| 47| 49| pus| 0.6690888|
 TrAP|TREATMENT| 240| 251| levofloxacin|PROBLEM| 341| 362| acanthamoeba sinusitis| 0.7076752|
 TrAP|TREATMENT| 45| 54| vancomycin|PROBLEM| 149| 190| enteric and non-enteric gram negative rods| 0.5275054|
 TrNAP| TEST| 17| 31| mediastinoscopy|PROBLEM| 185| 195| anthracosis| 0.507071|
 PIP| PROBLEM| 55| 96| an additional dysfunction on the left side|PROBLEM| 109| 140| wide-spread cortical dysfunction| 0.8733203|
 PIP| PROBLEM| 138| 171| anteroseptal myocardial infarction|PROBLEM| 93| 112| q waves in leads iii| 0.7825291|
 TrAP|TREATMENT| 167| 176| vancomycin|PROBLEM| 232| 241| gi sources| 0.5905505|
 TrNAP| PROBLEM| 182| 190| gastritis|PROBLEM| 45| 84| erosion of the gastroesophageal junction| 0.7097319|
 TeRP| TEST| 1| 17| an echocardiogram|PROBLEM| 35| 58| a dilated left ventricle| 0.5644851|
 TrAP|TREATMENT| 74| 84| clindamycin|PROBLEM| 34| 48| his throat pain| 0.8434692|
 TrAP|TREATMENT| 56| 65| management|PROBLEM| 70| 91| the increased bleeding|0.64535075|
 TeRP| TEST| 1| 11| chest x-ray|PROBLEM| 29| 38| infiltrate| 0.7333719|
 TeRP| TEST| 1| 15| his chest x-ray|PROBLEM| 23| 61| diffuse patchy infiltrates on left lung|0.71429074|
 PIP| PROBLEM| 93| 120| a left lower lobe infiltrate|PROBLEM| 124| 132| pneumonia|0.69691706|
 TeRP| TEST| 1| 30| a computerized tomography scan|PROBLEM| 138| 183| proximal middle cerebral artery stem occlusion|0.53247863|
 TrNAP| PROBLEM| 182| 190| gastritis|PROBLEM| 23| 41| barrett's esophagus|0.50695926|
 TeRP| TEST| 1| 9| a ct scan|PROBLEM| 77| 94| sphenoid sinusitis| 0.5825746|
 PIP| PROBLEM| 89| 111| duke &apos;s grade b-ii|PROBLEM| 63| 77| colon carcinoma|0.74584115|
 TeRP| TEST| 1| 17| an ech

In [0]:
result_df.limit(20).toPandas()

Out[59]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,PIP,PROBLEM,196,239,an inferior and right ventricular infarction,PROBLEM,145,176,1-mm st depressions in i and avl,0.85219336
1,TeRP,TEST,1,20,abdominal ultrasound,PROBLEM,54,71,gallbladder sludge,0.58730906
2,TrAP,TREATMENT,99,133,ir placement of a drainage catheter,PROBLEM,139,173,his abdominopelvic fluid collection,0.8456566
3,TeRP,TEST,1,13,a colonoscopy,PROBLEM,44,51,bleeding,0.51441264
4,TeRP,TEST,1,8,a ct abd,PROBLEM,16,36,8mm obstructing stone,0.7644676
5,TeRP,TEST,1,9,pathology,PROBLEM,19,77,poorly differentiated squamous cell carcinoma of the cervix,0.63362217
6,TrAP,TREATMENT,96,99,cpap,PROBLEM,57,76,abdominal distention,0.59488845
7,TrNAP,TREATMENT,119,126,diuresis,PROBLEM,1,20,creatinine elevation,0.59645915
8,PIP,PROBLEM,70,89,right hydronephrosis,PROBLEM,3,52,an 8 x 7 x 7 mm stone in the proximal right ureter,0.5685163
9,TrAP,TREATMENT,10,15,repair,PROBLEM,20,32,neck fracture,0.58386457


## Load trained model from disk

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

clinical_ner_tagger = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "tokens", "embeddings")\
    .setOutputCol("ner_tags") 

loaded_re_Model = RelationExtractionModel.load("dbfs:/databricks/driver/models/custom_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"]) \
    .setOutputCol("relations")\
    .setRelationPairs(["problem-test", "problem-treatment"])\
    .setPredictionThreshold(0.9)\
    .setMaxSyntacticDistance(4)

trained_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    loaded_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

loaded_re_model = trained_pipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]

In [0]:

text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

loaded_re_model_light = LightPipeline(loaded_re_model)
annotations = loaded_re_model_light.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.relation!="O"]

Out[62]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,TrAP,TREATMENT,512,522,amoxicillin,PROBLEM,528,556,a respiratory tract infection,0.9293416


End of Notebook #